In [192]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score
from sklearn.metrics import precision_score, recall_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import LabelEncoder, StandardScaler

wine = pd.read_csv("cleansingWine.csv")



<positron-console-cell-192>:12: DtypeWarning: Columns (8,19,20) have mixed types. Specify dtype option on import or set low_memory=False.


### Predicting Price

In [174]:
# Select Variables to predict Price
features = ['varieties1', 'abv', 'degree', 'sweet', 'acidity', 'body', 'tannin', 'year']
target = 'price'

wine_data_clean = wine.dropna(subset=[target])

# Convert categorical features into numeric representations using LabelEncoder
label_encoders = {}
for col in features:
    if wine_data_clean[col].dtype == 'object':  
        le = LabelEncoder()
        wine_data_clean[col] = le.fit_transform(wine_data_clean[col].fillna('Unknown'))
        label_encoders[col] = le

wine_data_clean = wine_data_clean.dropna(subset=features)

X = wine_data_clean[features]
y = wine_data_clean[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [180]:
# Build the first model
inputs = keras.Input(shape=(X_train_scaled.shape[1],))
x = layers.Dense(64, activation='relu')(inputs)
x = layers.Dense(32, activation='relu')(x)
x = layers.Dense(16, activation='relu')(x)
outputs = layers.Dense(1)(x)

model1 = keras.Model(inputs=inputs, outputs=outputs, name="model_1")
model1.compile(optimizer='adam', loss='mse', metrics=['mae'])

history1 = model1.fit(X_train_scaled, y_train, validation_split=0.1, epochs=50, batch_size=32, verbose=0)

y_pred1 = model1.predict(X_test_scaled).flatten()

mae_1 = mean_absolute_error(y_test, y_pred1)
rmse_1 = np.sqrt(mean_squared_error(y_test, y_pred1))
r2_1 = r2_score(y_test, y_pred1)

print(f"Model 1 - MAE: {mae_1:.2f}, RMSE: {rmse_1:.2f}, R²: {r2_1:.4f}")


  1/136 ━━━━━━━━━━━━━━━━━━━━ 6s 46ms/step

101/136 ━━━━━━━━━━━━━━━━━━━━ 0s 501us/step

136/136 ━━━━━━━━━━━━━━━━━━━━ 0s 770us/step
136/136 ━━━━━━━━━━━━━━━━━━━━ 0s 770us/step


Model 1 - MAE: 99517.93, RMSE: 249835.02, R²: 0.0820
Model 1 - MAE: 99517.93, RMSE: 249835.02, R²: 0.0820


In [181]:
# Build the second model
inputs = keras.Input(shape=(X_train_scaled.shape[1],))
x = layers.Dense(128, activation='relu')(inputs)
x = layers.Dropout(0.3)(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dense(32, activation='relu')(x)
outputs = layers.Dense(1)(x)

model2 = keras.Model(inputs=inputs, outputs=outputs, name="model_2")
model2.compile(optimizer='adam', loss='mse', metrics=['mae'])

history2 = model2.fit(X_train_scaled, y_train, validation_split=0.1, epochs=50, batch_size=32, verbose=0)

y_pred2 = model2.predict(X_test_scaled).flatten()

mae_2 = mean_absolute_error(y_test, y_pred2)
rmse_2 = np.sqrt(mean_squared_error(y_test, y_pred2))
r2_2 = r2_score(y_test, y_pred2)

print(f"Model 2 - MAE: {mae_2:.2f}, RMSE: {rmse_2:.2f}, R²: {r2_2:.4f}")


  1/136 ━━━━━━━━━━━━━━━━━━━━ 5s 39ms/step

125/136 ━━━━━━━━━━━━━━━━━━━━ 0s 405us/step

136/136 ━━━━━━━━━━━━━━━━━━━━ 0s 693us/step
136/136 ━━━━━━━━━━━━━━━━━━━━ 0s 693us/step


Model 2 - MAE: 98499.49, RMSE: 249641.74, R²: 0.0834
Model 2 - MAE: 98499.49, RMSE: 249641.74, R²: 0.0834


In [187]:
# Build the third model
inputs = keras.Input(shape=(X_train_scaled.shape[1],))
x = layers.Dense(64, activation='relu')(inputs)
x = layers.Dense(32, activation='relu')(x)
x = layers.Dense(16, activation='relu')(x)
outputs = layers.Dense(1)(x)

model3 = keras.Model(inputs=inputs, outputs=outputs, name="model_3")
model3.compile(optimizer='adam', loss='log_cosh', metrics=['mae'])

history3 = model3.fit(X_train_scaled, y_train, validation_split=0.1, epochs=50, batch_size=32, verbose=0)

y_pred3 = model3.predict(X_test_scaled).flatten()

mae_3 = mean_absolute_error(y_test, y_pred3)
rmse_3 = np.sqrt(mean_squared_error(y_test, y_pred3))
r2_3 = r2_score(y_test, y_pred3)

print(f"Model 3 - MAE: {mae_3:.2f}, RMSE: {rmse_3:.2f}, R²: {r2_3:.4f}")


  1/136 ━━━━━━━━━━━━━━━━━━━━ 4s 32ms/step

136/136 ━━━━━━━━━━━━━━━━━━━━ 0s 387us/step

136/136 ━━━━━━━━━━━━━━━━━━━━ 0s 446us/step
136/136 ━━━━━━━━━━━━━━━━━━━━ 0s 446us/step


Model 3 - MAE: 86001.12, RMSE: 259091.83, R²: 0.0127
Model 3 - MAE: 86001.12, RMSE: 259091.83, R²: 0.0127


The second model preformed the best out of these three. It uses a neural network architecture with three hidden layers (128, 64, and 32 neurons) and dropout regularization to prevent overfitting. It's trained using the Adam optimizer to minimize mean squared error over 50 iterations.

### Predicting Nation

In [188]:
# Change Variables to Predict Nation
features = ['varieties1', 'abv', 'degree', 'sweet', 'acidity', 'body', 'tannin', 'year', 'price']
target = 'nation'

wine_data_clean = wine.dropna(subset=[target])

# Encode categorical features
label_encoders = {}
for col in features:
    if wine_data_clean[col].dtype == 'object':
        le = LabelEncoder()
        wine_data_clean[col] = le.fit_transform(wine_data_clean[col].fillna('Unknown'))
        label_encoders[col] = le

target_encoder = LabelEncoder()
wine_data_clean[target] = target_encoder.fit_transform(wine_data_clean[target])
wine_data_clean.dropna(subset=features, inplace=True)

# Split into features and labels
X = wine_data_clean[features]
y = wine_data_clean[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



<positron-console-cell-188>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<positron-console-cell-188>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<positron-console-cell-188>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<positron-console-cell-188>:13: SettingW

In [198]:
# Build the first model
inputs = keras.Input(shape=(X_train_scaled.shape[1],))
x = layers.Dense(64, activation='relu')(inputs)
x = layers.Dense(32, activation='relu')(x)
x = layers.Dense(16, activation='relu')(x)
outputs = layers.Dense(len(np.unique(y)), activation='softmax')(x)  # Multiclass output

model = keras.Model(inputs=inputs, outputs=outputs, name="wine_nation_classifier")

# Compile model
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=keras.optimizers.Adam(),
    metrics=['accuracy']
)

# Train the model
history = model.fit(X_train_scaled, y_train, validation_split=0.1, epochs=50, batch_size=32, verbose=0)

# Predict on test set
y_pred_probs = model.predict(X_test_scaled)
y_pred = np.argmax(y_pred_probs, axis=1)

# Evaluate performance
acc = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)

print(f"Set 1 - Accuracy: {acc:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")



  1/136 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step

136/136 ━━━━━━━━━━━━━━━━━━━━ 0s 406us/step

136/136 ━━━━━━━━━━━━━━━━━━━━ 0s 471us/step
136/136 ━━━━━━━━━━━━━━━━━━━━ 0s 471us/step


Set 1 - Accuracy: 0.5159, Precision: 0.4944, Recall: 0.5159
Set 1 - Accuracy: 0.5159, Precision: 0.4944, Recall: 0.5159


In [199]:
# Build the second model
inputs = keras.Input(shape=(X_train_scaled.shape[1],))
x = layers.Dense(32, activation='relu')(inputs)
x = layers.Dropout(0.3)(x)
x = layers.Dense(16, activation='relu')(x)
outputs = layers.Dense(len(np.unique(y)), activation='softmax')(x)  # Multiclass output

model2 = keras.Model(inputs=inputs, outputs=outputs, name="wine_nation_classifier_2")

model2.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy']
)

history2 = model2.fit(X_train_scaled, y_train, validation_split=0.1, epochs=50, batch_size=32, verbose=0)

y_pred_probs2 = model2.predict(X_test_scaled)
y_pred2 = np.argmax(y_pred_probs2, axis=1)

acc2 = accuracy_score(y_test, y_pred2)
precision2 = precision_score(y_test, y_pred2, average='weighted', zero_division=0)
recall2 = recall_score(y_test, y_pred2, average='weighted', zero_division=0)

print(f"Set 2 - Accuracy: {acc2:.4f}, Precision: {precision2:.4f}, Recall: {recall2:.4f}")

  1/136 ━━━━━━━━━━━━━━━━━━━━ 4s 30ms/step

136/136 ━━━━━━━━━━━━━━━━━━━━ 0s 373us/step
136/136 ━━━━━━━━━━━━━━━━━━━━ 0s 373us/step


Set 2 - Accuracy: 0.4592, Precision: 0.4115, Recall: 0.4592
Set 2 - Accuracy: 0.4592, Precision: 0.4115, Recall: 0.4592


In [200]:
# Build the third model
inputs = keras.Input(shape=(X_train_scaled.shape[1],))
x = layers.Dense(128, activation='relu')(inputs)
x = layers.BatchNormalization()(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dense(32, activation='relu')(x)
outputs = layers.Dense(len(np.unique(y)), activation='softmax')(x)  # Multiclass output

model3 = keras.Model(inputs=inputs, outputs=outputs, name="wine_nation_classifier_3")

model3.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=keras.optimizers.Adam(learning_rate=0.0005),
    metrics=['accuracy']
)

history3 = model3.fit(X_train_scaled, y_train, validation_split=0.1, epochs=50, batch_size=32, verbose=0)

y_pred_probs3 = model3.predict(X_test_scaled)
y_pred3 = np.argmax(y_pred_probs3, axis=1)

acc3 = accuracy_score(y_test, y_pred3)
precision3 = precision_score(y_test, y_pred3, average='weighted', zero_division=0)
recall3 = recall_score(y_test, y_pred3, average='weighted', zero_division=0)

print(f"Set 3 - Accuracy: {acc3:.4f}, Precision: {precision3:.4f}, Recall: {recall3:.4f}")

  1/136 ━━━━━━━━━━━━━━━━━━━━ 6s 46ms/step

136/136 ━━━━━━━━━━━━━━━━━━━━ 0s 572us/step

136/136 ━━━━━━━━━━━━━━━━━━━━ 0s 641us/step
136/136 ━━━━━━━━━━━━━━━━━━━━ 0s 641us/step


Set 3 - Accuracy: 0.5707, Precision: 0.5644, Recall: 0.5707
Set 3 - Accuracy: 0.5707, Precision: 0.5644, Recall: 0.5707


The third model performs the best. This model uses a deeper architecture with batch normalization layers and a lower learning rat. Its improved accuracy, precision, and recall suggest that the additional layers and normalization helped the network better learn patterns for predicting the nation a wine label comes from.